In [1]:
from sklearn.naive_bayes import MultinomialNB # ideal für counting features wie bow oder tfidf https://towardsdatascience.com/why-how-to-use-the-naive-bayes-algorithms-in-a-regulated-industry-with-sklearn-python-code-dbd8304ab2cf
from sklearn.naive_bayes import GaussianNB # für Features in Decimal Form geeignet
from sklearn.naive_bayes import ComplementNB # ähnlich wie Multinomial, soll sich aber besser für imbalanced data eignen
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    recall_score,
    precision_score,
    classification_report,
)
import vectorize_functions
import pandas as pd

In [2]:
def evaluate(y_test,y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    print("F1 Score:", f1)
    print("Recall:", recall)
    print("Precision:", precision)
    print(pd.DataFrame(confusion_matrix(y_test, y_pred)))

In [3]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = vectorize_functions.vectorize_bow()
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = vectorize_functions.vectorize_tfidf()
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = vectorize_functions.vectorize_w2v()
X_train_ft, X_test_ft, y_train_ft, y_test_ft = vectorize_functions.vectorize_ft()

In [4]:
print(X_train_bow.shape)
print(X_test_bow.shape)
print(y_train_bow.shape)
print(y_test_bow.shape)
print(type(X_train_bow))

(13737, 4925)
(5888, 4925)
(13737,)
(5888,)
<class 'scipy.sparse._csr.csr_matrix'>


In [5]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)
print(y_train_tfidf.shape)
print(y_test_tfidf.shape)
print(type(X_train_tfidf))

(19354, 6293)
(8295, 6293)
(19354,)
(8295,)
<class 'scipy.sparse._csr.csr_matrix'>


In [6]:
print(X_train_w2v.shape)
print(X_test_w2v.shape)
print(y_train_w2v.shape)
print(y_test_w2v.shape)
print(type(X_train_w2v))

(19354, 300)
(8295, 300)
(19354,)
(8295,)
<class 'numpy.ndarray'>


In [7]:
print(X_train_ft.shape)
print(X_test_ft.shape)
print(y_train_ft.shape)
print(y_test_ft.shape)
print(type(X_train_ft))

(19354, 300)
(8295, 300)
(19354,)
(8295,)
<class 'numpy.ndarray'>


 #### BoW

In [42]:
clf_bow = MultinomialNB()
clf_bow.fit(X_train_bow,y_train_bow)
y_pred_bow = clf_bow.predict(X_test_bow)

In [43]:
evaluate(y_test_bow, y_pred_bow)

Accuracy: 0.9410488245931284
F1 Score: 0.5852417302798982
Recall: 0.6377079482439926
Precision: 0.5407523510971787
      0    1
0  7461  293
1   196  345


In [49]:
clf_bow_comp = ComplementNB()
clf_bow_comp.fit(X_train_bow,y_train_bow)
y_pred_bow_comp = clf_bow_comp.predict(X_test_bow)

In [50]:
evaluate(y_test_bow, y_pred_bow_comp)

Accuracy: 0.8546112115732369
F1 Score: 0.4300567107750472
Recall: 0.8410351201478743
Precision: 0.28888888888888886
      0     1
0  6634  1120
1    86   455


#### TF-IDF

In [46]:
clf_tfidf = MultinomialNB()
clf_tfidf.fit(X_train_tfidf,y_train_tfidf)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

In [47]:
evaluate(y_test_tfidf, y_pred_tfidf)

Accuracy: 0.9482820976491863
F1 Score: 0.3529411764705882
Recall: 0.21626617375231053
Precision: 0.9590163934426229
      0    1
0  7749    5
1   424  117


In [51]:
clf_tfidf_comp = ComplementNB()
clf_tfidf_comp.fit(X_train_tfidf,y_train_tfidf)
y_pred_tfidf_comp = clf_tfidf_comp.predict(X_test_tfidf)

In [52]:
evaluate(y_test_bow, y_pred_tfidf_comp)

Accuracy: 0.8603978300180832
F1 Score: 0.42157842157842157
Recall: 0.7800369685767098
Precision: 0.2888432580424367
      0     1
0  6715  1039
1   119   422


#### W2V

In [55]:
clf_w2v = GaussianNB()
clf_w2v.fit(X_train_w2v,y_train_w2v)
y_pred_w2v = clf_w2v.predict(X_test_w2v)

In [56]:
evaluate(y_test_w2v, y_pred_w2v)

Accuracy: 0.708499095840868
F1 Score: 0.2716867469879518
Recall: 0.833641404805915
Precision: 0.16228859301907161
      0     1
0  5426  2328
1    90   451


#### FastText

In [57]:
clf_ft = GaussianNB()
clf_ft.fit(X_train_ft,y_train_ft)
y_pred_ft = clf_ft.predict(X_test_ft)

In [58]:
evaluate(y_test_ft, y_pred_ft)

Accuracy: 0.747799879445449
F1 Score: 0.29704301075268813
Recall: 0.8170055452865065
Precision: 0.1815195071868583
      0     1
0  5761  1993
1    99   442


## Tests mit modifizierter Vektorisierung

### Word2Vec

#### Vektoren summieren anstelle des Mittelwerts

In [78]:
from nltk import word_tokenize
from gensim.models import Word2Vec
import numpy as np
from gensim.models import FastText
from sklearn.model_selection import train_test_split

def vectorize_w2v_sum():
    df = pd.read_csv('../../../data/twitter_hate-speech/train_cleaned.csv', index_col=0)
    df = df[df['tweet_cleaned'].notna()]

    X_base = df.tweet_cleaned
    y_base = df.label

    X_train_base, X_test_base, y_train_base_sum, y_test_base_sum = train_test_split(X_base, y_base, test_size=0.3,
                                                                            random_state=42)

    X_train_base_tokenized = X_train_base.map(word_tokenize)
    X_test_base_tokenized = X_test_base.map(word_tokenize)

    w2v = Word2Vec(min_count=1, window=35, vector_size=300, sg=0)
    w2v.build_vocab(X_train_base_tokenized)#, progress_per=10000)
    w2v.train(X_train_base_tokenized, total_examples=len(X_train_base_tokenized), epochs=30)

    def w2v_vector(tokenized_tweet, size):
        vec = np.zeros(size).reshape((1, size))
        count = 0
        for word in tokenized_tweet:
            try:
                vec += w2v.wv[word].reshape((1, size))
                count += 1
            except KeyError:

                continue
        #if count != 0:
         #   vec /= count
        return vec

    size = 300
    X_train_w2v_sum = np.zeros((len(X_train_base_tokenized), size))
    for i in range(len(X_train_base_tokenized)):
        X_train_w2v_sum[i, :] = w2v_vector(X_train_base_tokenized.iloc[i], size)

    X_test_w2v_sum = np.zeros((len(X_test_base_tokenized), size))
    for i in range(len(X_test_base_tokenized)):
        X_test_w2v_sum[i, :] = w2v_vector(X_test_base_tokenized.iloc[i], size)

    return X_train_w2v_sum, X_test_w2v_sum, y_train_base_sum, y_test_base_sum


In [79]:
X_train_w2v_sum, X_test_w2v_sum, y_train_w2v_sum, y_test_w2v_sum = vectorize_w2v_sum()

In [81]:
clf_w2v_sum = GaussianNB()
clf_w2v_sum.fit(X_train_w2v_sum,y_train_w2v_sum)
y_pred_w2v_sum = clf_w2v_sum.predict(X_test_w2v_sum)

In [82]:
evaluate(y_test_w2v_sum, y_pred_w2v_sum)

Accuracy: 0.4412296564195298
F1 Score: 0.17658553917214426
Recall: 0.9186691312384473
Precision: 0.09768081761006289
      0     1
0  3163  4591
1    44   497


Recall höher als in Durchschnittsvariante, alle anderen schlechter

#### Vorhandenes Twitter Wörterbuch verwenden

In [ ]:
# Glove 200: https://nlp.stanford.edu/projects/glove/

In [5]:
from nltk import word_tokenize
from gensim.models import Word2Vec
import numpy as np
from gensim.models import FastText
from sklearn.model_selection import train_test_split
import gensim.downloader as api

def vectorize_glv():
    df = pd.read_csv('../../../data/twitter_hate-speech/train_cleaned.csv', index_col=0)
    df = df[df['tweet_cleaned'].notna()]

    X_base = df.tweet_cleaned
    y_base = df.label

    X_train_base, X_test_base, y_train_base_glv, y_test_base_glv = train_test_split(X_base, y_base, test_size=0.3,
                                                                            random_state=42)

    X_train_base_tokenized = X_train_base.map(word_tokenize)
    X_test_base_tokenized = X_test_base.map(word_tokenize)

    w2v = api.load("glove-twitter-200") #Word2Vec(min_count=1, window=35, vector_size=300, sg=0)
    #w2v.build_vocab(X_train_base_tokenized)#, progress_per=10000)
    #w2v.train(X_train_base_tokenized, total_examples=len(X_train_base_tokenized), epochs=30)

    def w2v_vector(tokenized_tweet, size):
        vec = np.zeros(size).reshape((1, size))
        count = 0
        for word in tokenized_tweet:
            try:
                vec += w2v[word].reshape((1, size))
                count += 1
            except KeyError:

                continue
        if count != 0:
            vec /= count
        return vec

    size = 200
    X_train_w2v_glv = np.zeros((len(X_train_base_tokenized), size))
    for i in range(len(X_train_base_tokenized)):
        X_train_w2v_glv[i, :] = w2v_vector(X_train_base_tokenized.iloc[i], size)

    X_test_w2v_glv = np.zeros((len(X_test_base_tokenized), size))
    for i in range(len(X_test_base_tokenized)):
        X_test_w2v_glv[i, :] = w2v_vector(X_test_base_tokenized.iloc[i], size)

    return X_train_w2v_glv, X_test_w2v_glv, y_train_base_glv, y_test_base_glv


In [ ]:
X_train_glv, X_test_glv, y_train_glv, y_test_glv = vectorize_glv()

In [7]:
clf_glv = GaussianNB()
clf_glv.fit(X_train_glv,y_train_glv)
y_pred_glv = clf_glv.predict(X_test_glv)

In [8]:
evaluate(y_test_glv, y_pred_glv)

Accuracy: 0.8738111413043478
F1 Score: 0.4117181314330958
Recall: 0.6951871657754011
Precision: 0.2924634420697413
      0    1
0  4885  629
1   114  260


Schlechterer Wert für Recall, alle anderen besser als oben in W2V

### FastText

In [110]:
def vectorize_ft_sum():
    df = pd.read_csv('../../../data/twitter_hate-speech/train_cleaned.csv', index_col=0)
    df = df[df['tweet_cleaned'].notna()]

    X_base = df.tweet_cleaned
    y_base = df.label

    X_train_base, X_test_base, y_train_base_ft_sum, y_test_base_ft_sum = train_test_split(X_base, y_base, test_size=0.3,
                                                                            random_state=42)

    X_train_base_tokenized = X_train_base.map(word_tokenize)
    X_test_base_tokenized = X_test_base.map(word_tokenize)

    ft = FastText(window=35, min_count=1, vector_size=300)
    ft.build_vocab(corpus_iterable=X_train_base_tokenized)
    ft.train(corpus_iterable=X_train_base_tokenized, total_examples=len(X_train_base_tokenized), epochs=30)

    def ft_vector(tokenized_tweet, size):
        vec = np.zeros(size).reshape((1, size))
        count = 0
        for word in tokenized_tweet:
            try:
                vec += ft.wv[word].reshape((1, size))
                count += 1
            except KeyError:

                continue
        #if count != 0:
        #    vec /= count
        return vec

    size = 300
    X_train_ft_sum = np.zeros((len(X_train_base_tokenized), size))
    for i in range(len(X_train_base_tokenized)):
        X_train_ft_sum[i, :] = ft_vector(X_train_base_tokenized.iloc[i], size)

    X_test_ft_sum = np.zeros((len(X_test_base_tokenized), size))
    for i in range(len(X_test_base_tokenized)):
        X_test_ft_sum[i, :] = ft_vector(X_test_base_tokenized.iloc[i], size)

    return X_train_ft_sum, X_test_ft_sum, y_train_base_ft_sum, y_test_base_ft_sum


In [111]:
X_train_ft_sum, X_test_ft_sum, y_train_ft_sum, y_test_ft_sum = vectorize_ft_sum()

In [112]:
clf_ft_sum = GaussianNB()
clf_ft_sum.fit(X_train_ft_sum,y_train_ft_sum)
y_pred_ft_sum = clf_ft_sum.predict(X_test_ft_sum)

In [113]:
evaluate(y_test_ft_sum, y_pred_ft_sum)

Accuracy: 0.5057263411693791
F1 Score: 0.19513152728700434
Recall: 0.9186691312384473
Precision: 0.10915879639797936
      0     1
0  3698  4056
1    44   497


Recall besser, alle anderen schlechter als oben